<a href="https://colab.research.google.com/github/MHosseinHashemi/Roshan_Internship/blob/main/TB_Detection_TF_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
import io
import os
import gc
import cv2
import h5py
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import ipywidgets as widgets
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
from google.colab import files
from sklearn.utils import shuffle
from warnings import filterwarnings
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import load_model
from IPython.display import display,clear_output
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L, EfficientNetV2S
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Defining a function to create CNN model

In [ ]:
image_size = 512
EfficientNetV2S = EfficientNetV2S(weights='imagenet',include_top=False,input_shape=(image_size,image_size,3))
model = EfficientNetV2S.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(rate=0.5)(model)
model = tf.keras.layers.Dense(1,activation='sigmoid')(model)
model = tf.keras.models.Model(inputs=EfficientNetV2S.input, outputs = model)

82420632/82420632 [==============================] - 3s 0us/step


In [ ]:
opt = SGD(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer = opt, metrics= ['accuracy','Recall','Precision'])

In [ ]:
csv_logger = CSVLogger('/content/drive/MyDrive/Colab Notebooks/Tensorflow_Logs/EfficientNetV2S.csv', separator=",")
checkpoint = ModelCheckpoint("/content/drive/MyDrive/Colab Notebooks/Tensorflow_Logs/EfficientNetV2S.h5", monitor="val_recall", save_best_only=True, mode="auto", verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy',
                              factor = 0.5,
                              patience = 2,
                              min_delta = 0.001,
                              mode='auto',
                              verbose=1)

In [ ]:
def train_generator(TRAINING_DIR):

    train_datagen = ImageDataGenerator(rescale=1/255)
    train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                        batch_size=4,
                                                        class_mode='binary',
                                                        target_size=(512, 512))

    return train_generator

In [ ]:
def test_generator(TEST_DIR):

    test_datagen = ImageDataGenerator(rescale=1/255)
    test_generator = test_datagen.flow_from_directory(directory=TEST_DIR,
                                                      batch_size=4,
                                                      class_mode='binary',
                                                      target_size=(512,512))

    return test_generator

In [ ]:
Train_Dir = '/content/drive/MyDrive/Colab Notebooks/New_dataset/train'
Test_Dir = '/content/drive/MyDrive/Colab Notebooks/New_dataset/validation'

In [ ]:
train_generator = train_generator(TRAINING_DIR=Train_Dir)

Found 5600 images belonging to 2 classes.


In [ ]:
validation_generator = test_generator(TEST_DIR=Test_Dir)

Found 1400 images belonging to 2 classes.


In [ ]:
EPOCHS = 25

history_1 = model.fit(
        train_generator,
        steps_per_epoch=200,
        epochs=EPOCHS,
        verbose=1,
        validation_data = validation_generator,
        callbacks=[csv_logger,checkpoint,reduce_lr]
        )

Epoch 1/25
200/200 [==============================] - ETA: 0s - loss: 0.4324 - accuracy: 0.8388 - recall: 0.8223 - precision: 0.8460
Epoch 1: val_recall improved from inf to 1.00000, saving model to /content/drive/MyDrive/Colab Notebooks/Tensorflow_Logs/EfficientNetV2S.h5
200/200 [==============================] - 158s 527ms/step - loss: 0.4324 - accuracy: 0.8388 - recall: 0.8223 - precision: 0.8460 - val_loss: 0.9415 - val_accuracy: 0.5000 - val_recall: 1.0000 - val_precision: 0.5000 - lr: 0.0100
Epoch 2/25
200/200 [==============================] - ETA: 0s - loss: 0.2077 - accuracy: 0.9312 - recall: 0.9373 - precision: 0.9205
Epoch 2: val_recall improved from 1.00000 to 0.96714, saving model to /content/drive/MyDrive/Colab Notebooks/Tensorflow_Logs/EfficientNetV2S.h5
200/200 [==============================] - 101s 505ms/step - loss: 0.2077 - accuracy: 0.9312 - recall: 0.9373 - precision: 0.9205 - val_loss: 0.0604 - val_accuracy: 0.9821 - val_recall: 0.9671 - val_precision: 0.9971 - l